In [1]:
%reload_ext dotenv
%dotenv

In [2]:
import ocean_lib
import sys
import time

print(f"Using python:{sys.version}")
print(f"Using ocean_lib: {ocean_lib.__version__}")

Using python:3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 16:22:27) 
[GCC 9.3.0]
Using ocean_lib: 0.8.5


# 1. Create ocean instance

In [3]:
from ocean_lib.config import Config
from ocean_lib.ocean.ocean import Ocean
config = Config('./config-dataunion-mumbai.ini')
ocean = Ocean(config)

print(f"config.network_url = '{config.network_url}'")
print(f"config.block_confirmations = {config.block_confirmations.value}")
print(f"config.metadata_cache_uri = '{config.metadata_cache_uri}'")
print(f"config.provider_url = '{config.provider_url}'")

# Create Alice's wallet
import os
from ocean_lib.web3_internal.wallet import Wallet
alice_wallet = Wallet(
    ocean.web3,
    os.getenv('TEST_PRIVATE_KEY1'),
    config.block_confirmations,
    config.transaction_timeout,
)
print(f"alice_wallet.address = '{alice_wallet.address}'")

config.network_url = 'https://rpc-mumbai.maticvigil.com'
config.block_confirmations = 1
config.metadata_cache_uri = 'https://aquarius.dataunion.market'
config.provider_url = 'https://provider.dev.dataunion.market/'
alice_wallet.address = '0xdF1dEc52e602020E27B0644Ea0F584b6Eb5CE4eA'


# 2. Publish asset

In [4]:
# Publish DATA datatoken, mint tokens
from ocean_lib.web3_internal.currency import to_wei

DATA_datatoken = ocean.create_data_token('DATA1', 'DATA1', alice_wallet, blob=ocean.config.metadata_cache_uri)
DATA_datatoken.mint(alice_wallet.address, to_wei(100), alice_wallet)
print(f"DATA_datatoken.address = '{DATA_datatoken.address}'")

# Specify metadata & service attributes for Branin test dataset.
# It's specified using _local_ DDO metadata format; Aquarius will convert it to remote
# by removing `url` and adding `encryptedFiles` field.
DATA_metadata = {
    "main": {
        "type": "dataset",
        "files": [
      {
        "url": "https://raw.githubusercontent.com/trentmc/branin/main/branin.arff",
        "index": 0,
        "contentType": "text/text"
      }
    ],
    "name": "branin", "author": "Trent", "license": "CC0",
    "dateCreated": "2019-12-28T10:55:11Z"
    }
}
DATA_service_attributes = {
    "main": {
        "name": "DATA_dataAssetAccessServiceAgreement",
        "creator": alice_wallet.address,
        "timeout": 3600 * 24,
        "datePublished": "2019-12-28T10:55:11Z",
        "cost": 1.0, # <don't change, this is obsolete>
        }
    }

# Set up a service provider. We'll use this same provider for ALG
from ocean_lib.data_provider.data_service_provider import DataServiceProvider
provider_url = DataServiceProvider.get_url(ocean.config)


# Calc DATA service compute descriptor
from ocean_lib.services.service import Service
from ocean_lib.common.agreements.service_types import ServiceTypes
DATA_compute_service = Service(
    service_endpoint=provider_url,
    service_type=ServiceTypes.CLOUD_COMPUTE,
    attributes=DATA_service_attributes
)

DATA_access_service = Service(
    service_endpoint=provider_url,
    service_type=ServiceTypes.ASSET_ACCESS,
    attributes=DATA_service_attributes,
)

# Publish metadata and service info on-chain
DATA_ddo = ocean.assets.create(
  metadata=DATA_metadata, # {"main" : {"type" : "dataset", ..}, ..}
  publisher_wallet=alice_wallet,
  services=[DATA_compute_service, DATA_access_service],
  data_token_address=DATA_datatoken.address)
print(f"DATA did = '{DATA_ddo.did}'")

DATA_datatoken.address = '0xe471d09778A4110deD0e545218360C63a9659aF1'
signing message with nonce 0: 0xe471d09778A4110deD0e545218360C63a9659aF1, account=0xdF1dEc52e602020E27B0644Ea0F584b6Eb5CE4eA
DATA did = 'did:op:e471d09778A4110deD0e545218360C63a9659aF1'


# 3. Publish algorithm

In [5]:
# Publish ALG datatoken
ALG_datatoken = ocean.create_data_token('ALG1', 'ALG1', alice_wallet, blob=ocean.config.metadata_cache_uri)
ALG_datatoken.mint(alice_wallet.address, to_wei(100), alice_wallet)
print(f"ALG_datatoken.address = '{ALG_datatoken.address}'")

# Specify metadata and service attributes, for "GPR" algorithm script.
# In same location as Branin test dataset. GPR = Gaussian Process Regression.
ALG_metadata =  {
    "main": {
        "type": "algorithm",
        "algorithm": {
            "language": "python",
            "format": "docker-image",
            "version": "0.1",
            "container": {
              "entrypoint": "python $ALGO",
              "image": "oceanprotocol/algo_dockers",
              "tag": "python-branin"
            }
        },
        "files": [
      {
        "url": "https://raw.githubusercontent.com/trentmc/branin/main/gpr.py",
        "index": 0,
        "contentType": "text/text",
      }
    ],
    "name": "gpr", "author": "Trent", "license": "CC0",
    "dateCreated": "2020-01-28T10:55:11Z"
    }
}
ALG_service_attributes = {
        "main": {
            "name": "ALG_dataAssetAccessServiceAgreement",
            "creator": alice_wallet.address,
            "timeout": 3600 * 24,
            "datePublished": "2020-01-28T10:55:11Z",
            "cost": 1.0, # <don't change, this is obsolete>
        }
    }

# Calc ALG service access descriptor. We use the same service provider as DATA
ALG_access_service = Service(
    service_endpoint=provider_url,
    service_type=ServiceTypes.CLOUD_COMPUTE,
    attributes=ALG_service_attributes
)

# Publish metadata and service info on-chain
ALG_ddo = ocean.assets.create(
  metadata=ALG_metadata, # {"main" : {"type" : "algorithm", ..}, ..}
  publisher_wallet=alice_wallet,
  services=[ALG_access_service],
  data_token_address=ALG_datatoken.address)
print(f"ALG did = '{ALG_ddo.did}'")

ALG_datatoken.address = '0x435cbb5426e8748763DC272d12E45aFcCD49879c'
signing message with nonce 1: 0x435cbb5426e8748763DC272d12E45aFcCD49879c, account=0xdF1dEc52e602020E27B0644Ea0F584b6Eb5CE4eA
ALG did = 'did:op:435cbb5426e8748763DC272d12E45aFcCD49879c'


# 4. Alice allows the algorithm for C2D for that data asset

In [6]:
time.sleep(30) # wait until Aquarius caches the asset  
from ocean_lib.assets.trusted_algorithms import add_publisher_trusted_algorithm
add_publisher_trusted_algorithm(DATA_ddo, ALG_ddo.did, config.metadata_cache_uri)
ocean.assets.update(DATA_ddo, publisher_wallet=alice_wallet)

'0x7c9acfaa2014c775bf1df10ca6319a490f41cfdffd84fc4146c55a8206317bf9'

# 5. Bob acquires datatokens for data and algorithm

In [7]:
bob_wallet = Wallet(
    ocean.web3,
    os.getenv('TEST_PRIVATE_KEY2'),
    config.block_confirmations,
    config.transaction_timeout,
)
print(f"bob_wallet.address = '{bob_wallet.address}'")

# Alice shares access for both to Bob, as datatokens. Alternatively, Bob might have bought these in a market.
DATA_datatoken.transfer(bob_wallet.address, to_wei(5), from_wallet=alice_wallet)
ALG_datatoken.transfer(bob_wallet.address, to_wei(5), from_wallet=alice_wallet)

bob_wallet.address = '0x7F6470a7b08Bf4dfb3Ebd9330C5B3D594B1d5BB9'


'0x212c3b3345f2a249d09398594a926578687875e0008dd2429af920f468ef7ba9'

# 6. Bob starts a compute job

In [8]:
DATA_did = DATA_ddo.did  # for convenience
ALG_did = ALG_ddo.did
DATA_DDO = ocean.assets.resolve(DATA_did)  # make sure we operate on the updated and indexed metadata_cache_uri versions
ALG_DDO = ocean.assets.resolve(ALG_did)

compute_service = DATA_DDO.get_service('compute')
algo_service = ALG_DDO.get_service('access')

from ocean_lib.web3_internal.constants import ZERO_ADDRESS
from ocean_lib.models.compute_input import ComputeInput

# order & pay for dataset
dataset_order_requirements = ocean.assets.order(
    DATA_did, bob_wallet.address, service_type=compute_service.type
)
DATA_order_tx_id = ocean.assets.pay_for_service(
        ocean.web3,
        dataset_order_requirements.amount,
        dataset_order_requirements.data_token_address,
        DATA_did,
        compute_service.index,
        ZERO_ADDRESS,
        bob_wallet,
        dataset_order_requirements.computeAddress,
    )

# order & pay for algo
algo_order_requirements = ocean.assets.order(
    ALG_did, bob_wallet.address, service_type=algo_service.type
)
ALG_order_tx_id = ocean.assets.pay_for_service(
        ocean.web3,
        algo_order_requirements.amount,
        algo_order_requirements.data_token_address,
        ALG_did,
        algo_service.index,
        ZERO_ADDRESS,
        bob_wallet,
        algo_order_requirements.computeAddress,
)

compute_inputs = [ComputeInput(DATA_did, DATA_order_tx_id, compute_service.index)]
job_id = ocean.compute.start(
    compute_inputs,
    bob_wallet,
    algorithm_did=ALG_did,
    algorithm_tx_id=ALG_order_tx_id,
    algorithm_data_token=ALG_datatoken.address
)
print(f"Started compute job with id: {job_id}")

Started compute job with id: 3036245789d14fc1a92b08e846030402


# 7. Get output

In [9]:
ocean.compute.status(DATA_did, job_id, bob_wallet)

{'ok': True, 'status': 70, 'statusText': 'Job finished'}

In [10]:
result = ocean.compute.result_file(DATA_did, job_id, 0, bob_wallet)  # 0 index, means we retrieve the results from the first dataset index

import pickle
model = pickle.loads(result)  # the gaussian model result

Exception: b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'

In [12]:
DATA_did)
_, service_endpoint = ocean.compute._get_compute_result_file_endpoint(did)

NameError: name 'did' is not defined